In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from datetime import datetime
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
Fixed_Time = holo.hardening.Fixed_Time

mtot = 1e9 * MSOL
mrat = 0.3

rads = np.logspace(-4, 4, 100) * PC
rchar = 10.0 * PC
g1 = -1.0
g2 = +2.5
norm = 1.0

fig, axes = plot.figax(figsize=[8, 5], ncols=2)


rr = rads / rchar
xx = rads / PC

dadt_func = Fixed_Time.function(norm, rr, g1, g2)
dadt_tot = Fixed_Time._dadt_dedt(mtot, mrat, rads, norm, rchar, g1, g2)[0]

# ---- plot da/dt

ax = axes[0]

yy = - dadt
cc, = ax.plot(xx, yy, alpha=0.5)
yy = - dadt_tot
ax.plot(xx, yy, color=cc.get_color(), ls='--', lw=2.0, alpha=0.5)

# ---- plot tau

ax = axes[1]

yy = - rads/dadt/PC
cc, = ax.plot(xx, yy, alpha=0.5)
yy = - rads/dadt_tot/PC
ax.plot(xx, yy, color=cc.get_color(), ls='--', lw=2.0, alpha=0.5)

plt.show()

In [ ]:
def draw_powerlaws(ax, powerlaws=[1.0, 2.0, 3.0]):
    import zcode.plot as zplot
    
    for pl in powerlaws:
        l1, = ax.plot(xx, np.power(xx, pl), color='0.5', ls='--', alpha=0.25)    
        l2, = ax.plot(xx, np.power(xx, -pl), color='0.5', ls='--', alpha=0.25)    
        zplot.label_line(ax, l1, f"${pl:+.1f}$", x=0.1, flip_rotation=False, alpha=0.35)
        zplot.label_line(ax, l2, f"${-pl:+.1f}$", x=0.9, flip_rotation=False, alpha=0.35)

    return


Fixed_Time = holo.hardening.Fixed_Time

mtot = 1e9 * MSOL
mrat = 0.3

rads = np.logspace(-4, 4, 100) * PC
rchar = 100.0 * PC

g1_list = [-1.0, 0.0, +1.0]

g2 = +2.5
norm = 1.0

fig, axes = plot.figax(figsize=[10, 5], ncols=2, wspace=0.25)

rr = rads / rchar
xx = rads / PC

for g1 in g1_list:

    dadt_func = Fixed_Time.function(norm, rr, g1, g2)
    dadt_tot = Fixed_Time._dadt_dedt(mtot, mrat, rads, norm, rchar, g1, g2)[0]

    # ---- plot da/dt

    ax = axes[0]
    ax.set(title='Rate', ylabel='da/dt [pc/time]', xlabel='Separation [pc]')

    yy = - dadt_func
    cc, = ax.plot(xx, yy, alpha=0.5, label=f"${+g1:.1f}$")
    yy = - dadt_tot
    ax.plot(xx, yy, color=cc.get_color(), ls='--', lw=2.0, alpha=0.5)

    # ---- plot tau

    ax = axes[1]
    ax.set(title='Time', ylabel='Time [time]', xlabel='Separation [pc]')

    yy = - rads/dadt_func/PC
    cc, = ax.plot(xx, yy, alpha=0.5)
    yy = - rads/dadt_tot/PC
    ax.plot(xx, yy, color=cc.get_color(), ls='--', lw=2.0, alpha=0.5)

for ax in axes:
    draw_powerlaws(ax)
    ax.legend()

plt.show()